# English SDK for Apache Spark

## Initialization

In [2]:
from pyspark_ai import SparkAI

spark_ai=SparkAI(verbose=True)
spark_ai.activate() # active partial functions for Spark DataFrame

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/24 11:04:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Example 1: Auto sales by brand in US 2022

In [3]:
# Search and ingest web content into a DataFrame
# If you have set up google-api-python-client, you can just run
# auto_df = spark_ai.create_df("2022 USA national auto sales by brand")
auto_df = spark_ai.create_df("https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand")
auto_df.show()

INFO: Parsing URL: https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW auto_sales_2022 AS SELECT * FROM VALUES
('Toyota', 1849751, -9),
('Ford', 1767439, -2),
('Chevrolet', 1502389, 6),
('Honda', 881201, -33),
('Hyundai', 724265, -2),
('Kia', 693549, -1),
('Jeep', 684612, -12),
('Nissan', 682731, -25),
('Subaru', 556581, -5),
('Ram Trucks', 545194, -16),
('GMC', 517649, 7),
('Mercedes-Benz', 350949, 7),
('BMW', 332388, -1),
('Volkswagen', 301069, -20),
('Mazda', 294908, -11),
('Lexus', 258704, -15),
('Dodge', 190793, -12),
('Audi', 186875, -5),
('Cadillac', 134726, 14),
('Chrysler', 112713, -2),
('Buick', 103519, -42),
('Acura', 102306, -35),
('Volvo', 102038, -16),
('Mitsubishi', 102037, -16),
('Lincoln', 83486, -4),
('Porsche', 70065, 0),
('Genesis', 56410, 14),
('INFINITI', 46619, -20),
('MINI', 29504, -1),
('Alfa Romeo', 12845, -30),
('Maserati', 6413, -10),
('Bentley', 3975, 0),
('Lamborghini', 313

In [4]:
auto_df.ai.verify("expect sales change percentage to be between -100 to 100")

INFO: Generated code:
def check_sales_change_percentage(df) -> bool:
    from pyspark.sql.functions import col
    
    # Check if the sales_change_percentage column is between -100 and 100
    valid_percentage = df.filter((col("sales_change_percentage") >= -100) & (col("sales_change_percentage") <= 100))
    
    # Compare the count of valid_percentage rows with the total count of rows in the DataFrame
    if valid_percentage.count() == df.count():
        return True
    else:
        return False

result = check_sales_change_percentage(df)

INFO: 
Result: True


In [ ]:
auto_df.ai.plot()

INFO: To visualize the result of `df` using Plotly, you can follow these steps:

1. Convert the PySpark DataFrame to a Pandas DataFrame.
2. Create a Plotly figure using the Pandas DataFrame.
3. Display the plot directly.

Here's the code to achieve this:


```
import pandas as pd
import plotly.express as px

# Convert the PySpark DataFrame to a Pandas DataFrame
pdf = df.toPandas()

# Create a Plotly figure using the Pandas DataFrame
fig = px.bar(pdf, x='brand', y='us_sales', text='sales_change_percentage',
             labels={'us_sales': 'US Sales', 'sales_change_percentage': 'Sales Change Percentage'},
             title='Auto Sales 2022')

# Customize the plot
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# Display the plot directly
fig.show()
```

This code will create a bar chart with the `brand` on the x-axis, `us_sales` on the y-axis, and the `sales_change_percentage` displayed as text abo

In [7]:
auto_df.ai.plot("pie chart for US sales market shares, show the top 5 brands and the sum of others")

INFO: To visualize the result of `df` using Plotly, you can follow these steps:

1. Import the necessary libraries
2. Convert the PySpark DataFrame to a Pandas DataFrame
3. Calculate the top 5 brands and the sum of others
4. Create a pie chart using Plotly
5. Display the plot

Here's the code:


```
import plotly.graph_objs as go
import pandas as pd

# Convert the PySpark DataFrame to a Pandas DataFrame
pdf = df.toPandas()

# Calculate the top 5 brands and the sum of others
top_5_brands = pdf.nlargest(5, 'us_sales')
other_brands_sum = pdf.loc[~pdf.index.isin(top_5_brands.index), 'us_sales'].sum()

# Add the sum of others to the top 5 brands DataFrame
other_brands_row = pd.DataFrame({'brand': ['Others'], 'us_sales': [other_brands_sum]})
top_5_brands_with_others = pd.concat([top_5_brands, other_brands_row], ignore_index=True)

# Create a pie chart using Plotly
fig = go.Figure(data=[go.Pie(labels=top_5_brands_with_others['brand'],
                             values=top_5_brands_with_othe

In [8]:
# Apply transforms to a Dataframe
auto_top_growth_df=auto_df.ai.transform("brand with the highest growth")
auto_top_growth_df.show()

INFO: SQL query for the transform:
SELECT brand, sales_change_percentage
FROM temp_view_for_transform
ORDER BY sales_change_percentage DESC
LIMIT 1

+-------+-----------------------+
|  brand|sales_change_percentage|
+-------+-----------------------+
|Genesis|                     14|
+-------+-----------------------+



In [9]:
# Explain what a DataFrame is retrieving.
auto_top_growth_df.ai.explain()

'In summary, this dataframe is retrieving the brand with the highest sales change percentage from the `temp_view_for_transform` view. The result will show the brand and its corresponding sales change percentage.'

## Example 2: USA Presidents

In [8]:
# You can also specify the expected columns for the ingestion.
df=spark_ai.create_df("USA presidents", ["president", "vice_president"])
df.show()

INFO: Parsing URL: https://www.loc.gov/rr/print/list/057_chron.html

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW usa_presidents AS SELECT * FROM VALUES
('George Washington', 'John Adams'),
('John Adams', 'Thomas Jefferson'),
('Thomas Jefferson', 'Aaron Burr'),
('Thomas Jefferson', 'George Clinton'),
('James Madison', 'George Clinton'),
('James Madison', 'Elbridge Gerry'),
('James Monroe', 'Daniel D. Tompkins'),
('John Quincy Adams', 'John C. Calhoun'),
('Andrew Jackson', 'John C. Calhoun'),
('Andrew Jackson', 'Martin Van Buren'),
('Martin Van Buren', 'Richard M. Johnson'),
('William Henry Harrison', 'John Tyler'),
('John Tyler', 'office vacant'),
('James K. Polk', 'George M. Dallas'),
('Zachary Taylor', 'Millard Fillmore'),
('Millard Fillmore', 'office vacant'),
('Franklin Pierce', 'William R. King'),
('Franklin Pierce', 'office vacant'),
('James Buchanan', 'John C. Breckinridge'),
('Abraham Lincoln', 'Hannibal Hamlin'),
('Abraham Lincoln', 'Andrew Johnson'),
('Andre

In [9]:
presidents_who_were_vp = df.ai.transform("presidents who were also vice presidents")
presidents_who_were_vp.show()

INFO: SQL query for the transform:
SELECT DISTINCT president
FROM temp_view_for_transform
WHERE president IN (SELECT vice_president FROM temp_view_for_transform)

+------------------+
|         president|
+------------------+
|        John Adams|
|  Thomas Jefferson|
|  Martin Van Buren|
|  Millard Fillmore|
|        John Tyler|
|    Andrew Johnson|
| Chester A. Arthur|
|Theodore Roosevelt|
|   Calvin Coolidge|
|   Harry S. Truman|
|    Gerald R. Ford|
| Lyndon B. Johnson|
|  Richard M. Nixon|
|       George Bush|
|   Joseph R. Biden|
+------------------+



In [10]:
presidents_who_were_vp.ai.explain()

'In summary, this dataframe is retrieving the list of presidents who have never been vice-presidents from the `usa_presidents` table.'

In [11]:
presidents_who_were_vp.ai.verify("expect no NULL values")

INFO: Generated code:
def no_null_values(df) -> bool:
    from pyspark.sql.functions import col, count, when

    # Count the number of NULL values in each column
    null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

    # Check if there are any NULL values in any column
    for c in null_counts.columns:
        if null_counts.first()[c] > 0:
            return False

    return True

result = no_null_values(df)

INFO: 
Result: True


# Example 3: Top 10 tech companies

In [12]:
# Search and ingest web content into a DataFrame
company_df=spark_ai.create_df("Top 10 tech companies by market cap", ['company', 'cap', 'country'])
company_df.show()

INFO: Parsing URL: https://www.statista.com/statistics/1350976/leading-tech-companies-worldwide-by-market-cap/

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW top_tech_companies AS SELECT * FROM VALUES
('Apple', 2242, 'USA'),
('Microsoft', 1821, 'USA'),
('Alphabet (Google)', 1229, 'USA'),
('Amazon', 902.4, 'USA'),
('Tesla', 541.4, 'USA'),
('TSMC', 410.9, 'Taiwan'),
('NVIDIA', 401.7, 'USA'),
('Tencent', 377.8, 'China'),
('Meta Platforms (Facebook)', 302.1, 'USA'),
('Samsung', 301.7, 'South Korea')
AS v1(company, cap, country)

INFO: Storing data into temp view: top_tech_companies

+--------------------+------+-----------+
|             company|   cap|    country|
+--------------------+------+-----------+
|               Apple|2242.0|        USA|
|           Microsoft|1821.0|        USA|
|   Alphabet (Google)|1229.0|        USA|
|              Amazon| 902.4|        USA|
|               Tesla| 541.4|        USA|
|                TSMC| 410.9|     Taiwan|
|              NVID

In [13]:
us_company_df=company_df.ai.transform("companies in United States")
us_company_df.show()

INFO: SQL query for the transform:
SELECT * FROM temp_view_for_transform WHERE country = 'United States'

+-------+---+-------+
|company|cap|country|
+-------+---+-------+
+-------+---+-------+



In [14]:
us_company_df.ai.explain()

'In summary, this dataframe is retrieving the company, market capitalization (cap), and country for all the top tech companies that are located in the United States.'

In [15]:
us_company_df.ai.verify("expect all company names to be unique")

INFO: Generated code:
def has_unique_company_names(df) -> bool:
    from pyspark.sql.functions import countDistinct

    # Get the number of unique company names in the DataFrame
    unique_company_names = df.select(countDistinct("company")).collect()[0][0]

    # Get the total number of rows in the DataFrame
    total_rows = df.count()

    # Check if the number of unique company names is equal to the total number of rows
    if unique_company_names == total_rows:
        return True
    else:
        return False

result = has_unique_company_names(df)

INFO: 
Result: True


## Example 4: Ingestion from a URL
Instead of searching for the web page, you can also ask the SparkAI to ingest from a URL.

In [16]:
best_albums_df = spark_ai.create_df('https://time.com/6235186/best-albums-2022/', ["album", "artist", "year"])
best_albums_df.show()

INFO: Parsing URL: https://time.com/6235186/best-albums-2022/

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW best_albums_2022 AS SELECT * FROM VALUES
('Motomami', 'Rosalía', 2022),
('You Can’t Kill Me', '070 Shake', 2022),
('Mr. Morale & The Big Steppers', 'Kendrick Lamar', 2022),
('Big Time', 'Angel Olsen', 2022),
('Electricity', 'Ibibio Sound Machine', 2022),
('It’s Almost Dry', 'Pusha T', 2022),
('Chloe and the Next 20th Century', 'Father John Misty', 2022),
('Renaissance', 'Beyoncé', 2022),
('19 Masters', 'Saya Gray', 2022),
('Un Verano Sin Ti', 'Bad Bunny', 2022)
AS v1(album, artist, year)

INFO: Storing data into temp view: best_albums_2022

+--------------------+--------------------+----+
|               album|              artist|year|
+--------------------+--------------------+----+
|            Motomami|             Rosalía|2022|
|   You Can’t Kill Me|           070 Shake|2022|
|Mr. Morale & The ...|      Kendrick Lamar|2022|
|            Big Time|         An

In [17]:
best_albums_df.ai.verify("expect each year to be 2022")

INFO: Generated code:
def has_year_2022(df) -> bool:
    from pyspark.sql.functions import col

    # Filter the DataFrame to only include rows where the year is 2022
    df_filtered = df.filter(col("year") == 2022)

    # Check if the number of rows in the filtered DataFrame is equal to the number of rows in the original DataFrame
    if df_filtered.count() == df.count():
        return True
    else:
        return False

result = has_year_2022(df)

INFO: 
Result: True


## Example 5: UDF Generation

You can also ask the SparkAI to generate code for a Spark UDF by providing.

In [18]:
@spark_ai.udf
def convert_grades(grade_percent: float) -> str:
    """Convert the grade percent to a letter grade using standard cutoffs"""
    ...

INFO: Creating following Python UDF:
def convert_grades(grade_percent: float) -> str:
    if grade_percent is not None:
        if grade_percent >= 90:
            return "A"
        elif grade_percent >= 80:
            return "B"
        elif grade_percent >= 70:
            return "C"
        elif grade_percent >= 60:
            return "D"
        else:
            return "F"



In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.udf.register("convert_grades", convert_grades)
percentGrades = [(1, 97.8), (2, 72.3), (3, 81.2)]
df = spark.createDataFrame(percentGrades, ["student_id", "grade_percent"])
df.selectExpr("student_id", "convert_grades(grade_percent)").show()

+----------+-----------------------------+
|student_id|convert_grades(grade_percent)|
+----------+-----------------------------+
|         1|                            A|
|         2|                            C|
|         3|                            B|
+----------+-----------------------------+



# Cache
The SparkAI supports a simple in-memory and persistent cache system. It keeps an in-memory staging cache, which gets updated for LLM and web search results. The staging cache can be persisted through the commit() method. Cache lookup is always performed on both in-memory staging cache and persistent cache.

In [10]:
spark_ai.commit()